In [2]:
#r "nuget: SchlenkR.FsLocalState"

open System
open FsLocalState
open FsLocalState.Operators
open XPlot.Plotly

Installing package SchlenkR.FsLocalState...

In [3]:
type Sample =
    { pi: float
      point: float * float
      isInsideCircle: bool }

let monteCarlo =
    0 <|> fun lastInsideCount (_: unit) -> gen {
        let! samples = Gen.countFrom 1 1
        let! x = Gen.random()
        let! y = Gen.random()
        let distance = Math.Sqrt (x*x + y*y)
        let isInsideCircle = distance < 1.0
        let insideCount = if isInsideCircle then lastInsideCount + 1 else lastInsideCount

        let result =
            { pi = 4.0 * float insideCount / float samples
              point = x, y
              isInsideCircle = isInsideCircle }

        return { value = result
                 state = insideCount }
    }

In [4]:
let numSamples = 3_000

let piSeq = monteCarlo |> Gen.toSeq ignore
let calculatedResult = piSeq |> Seq.toListN numSamples

In [7]:
let calculatedPi = calculatedResult |> List.map (fun x -> x.pi)
let constPi = List.init numSamples (fun _ -> Math.PI)

[calculatedPi; constPi]
|> List.map (fun s -> Scatter(y = s))
|> Chart.Plot

In [8]:
let getPoints isInside =
    calculatedResult
    |> List.filter (fun x -> x.isInsideCircle = isInside)
    |> List.map (fun x -> x.point)
let insidePoints = getPoints true
let outsidePoints = getPoints false

[ insidePoints; outsidePoints ]
|> Chart.Scatter
|> Chart.WithSize (600, 600)
